In [8]:
import os
import json
import re
import pandas as pd
import numpy as np

from google.cloud import storage
from google.cloud import documentai_v1beta2 as documentai

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r"C:\Users\ksaluj1\Documents\work\dev\gv\iacoe_sa.json"
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/hsingh151/My_Work/Uber_IACOE_OCR_ML_Pipeline_v1.1.0/iacoe_sa.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/Users/jghosh2/Documents/my-notebook/BOL_CODE_REF/iacoe_sa.json'
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000

def parse_ner_json_to_string(parsed_ner):
    ner_string = ""
    flag = True
    for key, value in parsed_ner.items():
        if flag:
            ner_string = ner_string + value + "|" + key
            flag = False
        else:
            ner_string = ner_string + ";" + value + "|" + key
    return ner_string

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    file_name = source_file_name.split('/')[-1]
    destination_blob_name = destination_blob_name + "/" + file_name
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
    gs_bucket_url = "gs://iacoe-cloud-bucket"
    gcs_uri = gs_bucket_url + '/' + destination_blob_name
    return gcs_uri

def extract_data_doc_ai(input_uri, project_id='concrete-tuner-241417'):
         # input_uri='gs://iacoe-cloud-bucket/k'):
    """Process a single document with the Document AI API, including
    text extraction and entity extraction."""

    client = documentai.DocumentUnderstandingServiceClient()
    gcs_source = documentai.types.GcsSource(uri=input_uri)

    input_config = documentai.types.InputConfig(
        gcs_source=gcs_source, mime_type='application/pdf')

    form_extraction_params = documentai.types.FormExtractionParams(
        enabled=True)
    table_extraction_params = documentai.types.TableExtractionParams(
        enabled=True)
    entity_extraction_params = documentai.types.EntityExtractionParams(
        enabled=True)

    parent = 'projects/{}/locations/us'.format(project_id)
    request = documentai.types.ProcessDocumentRequest(
        parent=parent,
        input_config=input_config,
        form_extraction_params=form_extraction_params,
        table_extraction_params=table_extraction_params,
        entity_extraction_params=entity_extraction_params)

    document = client.process_document(request=request)
    data = {}
    data["raw_text"] = (document.text).encode('utf-8')

    def _get_text(el):
        """Convert text offset indexes into text snippets.
        """
        response = ''
        for segment in el.text_anchor.text_segments:
            start_index = segment.start_index
            end_index = segment.end_index
            response += document.text[start_index:end_index]
        return response
    ent_dict = {}
    for entity in document.entities:
        ent_dict[_get_text(entity)] = (entity.mention_text)

    data["entities"] = ent_dict
    form_ent = {}
    for page in document.pages:
        for form_field in page.form_fields:
            key = (_get_text(form_field.field_name), form_field.field_name.confidence)
            value = (_get_text(form_field.field_value), form_field.field_value.confidence)
            form_ent[key] = value
    data["form_data"] = form_ent
    
    out_tables = []
    for page in document.pages:
        for table_num, table in enumerate(page.tables):
            out_table = []
            for row_num, row in enumerate(table.header_rows):
                out_header_row = [_get_text(cell.layout) for cell in row.cells]
                out_table.append(out_header_row)
            for row_num, row in enumerate(table.body_rows):
                out_body_row = [_get_text(cell.layout) for cell in row.cells]
                out_table.append(out_body_row)
            out_tables.append(out_table)

    data["tables"] = out_tables
    
    labels = {}
    for label in document.labels:
        labels[label.name] = label.confidence
    data["labels"] = labels

    return data

In [9]:
pwd

'/Users/jghosh2/Documents/my-notebook/BOL_CODE_REF/02_NER_Bol_processing/02_Format_2/04_jupyter_notebook'

In [10]:
import time

In [11]:
data_files = {}

In [16]:
GCS_BUCKET = "iacoe-cloud-bucket"
DESTINATION_BLOB_NAME = "iacoe_ocr_ml"
file_loc = '/Users/jghosh2/Documents/my-notebook/BOL_CODE_REF/02_NER_Bol_processing/01_Format_1/Format1/pdf_samples/'

arr_txt =  [x for x in os.listdir(file_loc) if x.endswith(".pdf") or x.endswith(".PDF")]
t1 = time.time()

for i in arr_txt:
    file_path = file_loc + i
    t3 = time.time()
    gcs_uri = upload_blob(GCS_BUCKET, file_path, DESTINATION_BLOB_NAME)
    data_files[i] = extract_data_doc_ai(gcs_uri)
    print("File {} : time {}".format(i, time.time() - t3))
    
t2 = time.time()

File 48.pdf : time 8.834411144256592


In [17]:
print("total time to process {} files is {}".format(len(arr_txt), t2 - t1))

total time to process 1 files is 8.834857940673828


In [27]:
data_files[i]['raw_text'].decode()

"TRUCK#182142\nLOAD#0528065\nPepsi Beverages Company\nBILL OF LADING\nBOL#: OT4946069264\nFrom: 494 Mesquite, TX Plant\n4532 Highway 67 E.\nMesquite\nTX 75150-\nShip Date: 08/16/2021 14:51\nOrder ID:\nCustomer PO: 5824766113\nTo: 999 S8261 SAMS CLUB #18-8261\n455 31ST. STREET\nKENNER\nLA 70065\nArrival Date: 08/17/2021 04:00\nItem\nCS\n1\n1.0\nDescription\n88973 12OZ CN 36/1CB BRSK SWL\n19716 12OZ CN 36/1CB PEPSI\n106551 16.90Z PL PK 24/1 DT LPT GRN BRY\n95207 16.9OZ PL PK 24/1 DT LPT GT CIT\n95206 - 16.90Z PL PK 24/1 LIT GRN CTRS\n125294 16.90Z PL PK 24/1 LIT LMND\n144297 16.9OZ PL PK 24/1 LIT PCH\n95903 16.9OZ PLPK32/1 AQUA WTR\n82697 6.50Z CN 1/12 STRBK DS\n57933 9.5OZ NR 15/1 FRAP MCH\n120\n60\nCS\nShip\nPallet Extended\nPallets Qty UOMA Weight Weight\nComment\n2.0 150\n2,400\n4,800\n3.0 225 CS 2,355 7,065\n2.0 120\n1,665 3,330\n60 CS 1,790 1,790\n1.0\n60 CS 1,817 1,817\n2.0 120 CS 1,710 3,420\n2.0\nCS\n1,890\n3,780\n6.0 360 CS 2,278 13,669\n0.6 180 CS 1,710 1,026\n1.0 126\n2,129 2

In [41]:
data_files[i]['entities']

{'BOL#: OT4946069264': 'OT4946069264',
 'Ship Date: 08/16/2021 14:51': '08/16/2021 14:51',
 'Customer PO: 5824766113': '5824766113',
 'To: 999 S8261 SAMS CLUB #18-8261\n455 31ST. STREET': '999 S8261 SAMS CLUB #18-8261\n455 31ST. STREET',
 'KENNER\nLA 70065': 'LA 70065',
 'Arrival Date: 08/17/2021 04:00': '08/17/2021 04:00',
 'Total: 44,275': '44,275',
 'Grand Total:\n21.0 1,542.0': '21.0 1,542.0',
 'Received By\n44,275\nDate 8-17-21\nManer TR #': '44,275\nDate 8-17-21\nManer TR #',
 'Trailer #: Prointment.\nUNKNOWN Pale\n3482816\nDate': 'Prointment.\nUNKNOWN Pale\n3482816\nDate',
 'Checked By:\nBad Bad\nPallet Comment!\nthrich Good': 'Bad Bad\nPallet Comment!\nthrich Good',
 'Driver Signature:\nES': 'ES',
 'PO 582476643': '582476643',
 "Total Rec'd 21413² 1521": '21413² 1521',
 'BOL Comment: TR# 6250': 'TR# 6250',
 'Carrier: Common Carrier': 'Common Carrier',
 'Loaded By: Torres, Jose J.': 'Torres, Jose J.',
 'Driver Name: UNASSIGNED': 'UNASSIGNED',
 'Print Date: 08/16/2021 14:52': '08

In [66]:
def parse_ner_json_to_string(parsed_ner):
    ner_string = ""
    flag = True
    #print(parsed_ner)
    for key, value in parsed_ner.items():
        if flag:
            ner_string = ner_string + value + "|" + key
            flag = False
        else:
            ner_string = ner_string + ";" + value + "|" + key
    print(ner_string)        
    return ner_string

def pepsi_format1_doc_ai_ner_parser(raw_text, entities_json):
    parsed_data = {}
    
    #Look up keys
    BOL_KEY = "BOL#:"
    BOL_KEY2 = "BOL #:"
    DROPOFF_LOCATION_KEY = "To:"
    PICK_UP_ADDRESS_KEY = "From:"
    
    # Entities to return
    PO_NUMBER = "PO Number"
    DROPOFF_ADDRESS = "Dropoff Address"
    PICKUP_ADDRESS = "Pickup Address"
    
    PO_FLAG = True
    PICKUP_FLAG = True
    DROPOFF_FLAG = True
    
    for key, value in entities_json.items():
        
        if key.startswith(BOL_KEY) or key.startswith(BOL_KEY2):
            #print(key,value)
            if PO_FLAG:
                parsed_data[PO_NUMBER] = value
                #print(parsed_data)
                #print('**********************')
                PO_FLAG = False
                
    def get_zip_from_raw_text(text):
        pattern = re.compile(r'([A-Z]{2}\s\d{5})')
        match = pattern.findall(text.replace('\n', ' '))
        #print(match)
        if match:
            return match
        else:
            return ""
        
            
    def get_bol_number_from_raw_text(text):
        search_key = "BOL"
        start_idx = text.find(search_key)
        end_idx = text.find('From')
        bol_str = text[start_idx + len(search_key):end_idx].strip()
        try:
            bol = bol_str.strip()
            #print(bol)
            #print('***')
        except:
            bol = ''
            
        return bol
    
    def get_pickup_address_from_raw_text(text):
        zip_ = get_zip_from_raw_text(raw_text)
        search_key = "From"
        start_idx = text.find(search_key)
        end_idx = text.find('Ship ')
        pickup_str = text[start_idx + len(search_key):end_idx].strip()
        pickup = pickup_str.strip()
        pickup = pickup.replace('\n', ' ')
        pickup = pickup.replace(':', '')
        zip_check_pickup = get_zip_from_raw_text(pickup)    
        if len(pickup) < 85 and zip_check_pickup:
            return pickup
        else:
            return pickup[0:49] + ' ' +zip_[0]
        
    def get_dropoff_address_from_raw_text(text):
        zip_ = get_zip_from_raw_text(raw_text)
        search_key = "To:"
        start_idx = text.find(search_key)
        end_idx = text.find('Arrival')
        dropoff_str = text[start_idx + len(search_key):end_idx].strip()
        dropoff = dropoff_str.strip()
        dropoff = dropoff.replace('\n', ' ')
        zip_check_dropoff = get_zip_from_raw_text(dropoff)
        if dropoff and zip_check_dropoff:
            return dropoff
        else:
            return dropoff + ' ' + zip_[1]
        
        try:
            dropoff = dropoff_str.strip()
            dropoff = dropoff.replace('\n', ' ')
        except:
            dropoff = ''
            
        return dropoff 
           
    
    if PO_FLAG:
        bol_num = get_bol_number_from_raw_text(raw_text)
        bol_num = bol_num.replace('#: ', '')
        parsed_data[PO_NUMBER] = bol_num
        
    if PICKUP_FLAG:
        pickup_add = get_pickup_address_from_raw_text(raw_text)
        parsed_data[PICKUP_ADDRESS] = pickup_add
        
    if DROPOFF_FLAG:
        dropadd = get_dropoff_address_from_raw_text(raw_text)
        parsed_data[DROPOFF_ADDRESS] = dropadd
    
    #print(parsed_data)        
    ner_string = parse_ner_json_to_string(parsed_data)
    return [200, ner_string]
    #return parsed_data
    

In [67]:
a = pepsi_format1_doc_ai_ner_parser(data_files[i]['raw_text'].decode(), data_files[i]['entities'])

OT4946069264|PO Number; 494 Mesquite, TX Plant 4532 Highway 67 E. Mesquite TX 75150-|Pickup Address;999 S8261 SAMS CLUB #18-8261 455 31ST. STREET KENNER LA 70065|Dropoff Address


In [68]:
extract_data = []

In [69]:
for file,data in data_files.items():
    #print(file,data)
    a = pepsi_format1_doc_ai_ner_parser(data['raw_text'].decode(), data['entities'])
    extract_data.append(a)

53709863
(402) 32644363889
FOB:
Dispatch: 10/26/20 2:49 PM
SHIP TO
Name: AMAZON.COM SORT MORENO ONTS
Carrier Name: PEPSI LOGISTICS CO INC
MOS: T
Address: 24300 NANDINA AVE
Trailer Number: LIVER1071PLCB
City/State/Zip: MORENO VALLEY, CA 92551-9534
Seal number(s): 505732
SCAC: PLCB
CAR MOVE: 53709863
ID: C10817596
FOB:
Pro Number:
LOAD SEQ:
THIRD PARTY FREIGHT CHARGES BILL TO:
Freight Charge freight charges are prepaid unless marked
Name: PEPSICO CIO CASS INFO. SERVICES
otherwise)
Address: PO BOX 17608
Prepaid
Collect
3rd Party
City/State/Zip: ST LOUIS, MO 63178-7608
Master Bill of Lading with attached underlying Bils of
(check box) Lading
SPECIAL INSTRUCTIONS: OTHERS-All deliveries need to be scheduled through Amazon Carrier Central For TL- Enter ETMS load ID in PRO andBOL fiel ds. For LTL- Enter truePRO number 
in PRO field. For Reverse Logistics call 877-402-2435.
CUSTOMER ORDER NUMBER
# PKGS
CUSTOMER ORDER INFORMATION
Pallet/Slip
WEIGHT (LB)
(Circle One
11891 Y N
ADDITIONAL SHIPPER I

In [70]:
df = pd.DataFrame(extract_data)

In [71]:
extract_data

[[200,
  '53709863\n(402) 32644363889\nFOB:\nDispatch: 10/26/20 2:49 PM\nSHIP TO\nName: AMAZON.COM SORT MORENO ONTS\nCarrier Name: PEPSI LOGISTICS CO INC\nMOS: T\nAddress: 24300 NANDINA AVE\nTrailer Number: LIVER1071PLCB\nCity/State/Zip: MORENO VALLEY, CA 92551-9534\nSeal number(s): 505732\nSCAC: PLCB\nCAR MOVE: 53709863\nID: C10817596\nFOB:\nPro Number:\nLOAD SEQ:\nTHIRD PARTY FREIGHT CHARGES BILL TO:\nFreight Charge freight charges are prepaid unless marked\nName: PEPSICO CIO CASS INFO. SERVICES\notherwise)\nAddress: PO BOX 17608\nPrepaid\nCollect\n3rd Party\nCity/State/Zip: ST LOUIS, MO 63178-7608\nMaster Bill of Lading with attached underlying Bils of\n(check box) Lading\nSPECIAL INSTRUCTIONS: OTHERS-All deliveries need to be scheduled through Amazon Carrier Central For TL- Enter ETMS load ID in PRO andBOL fiel ds. For LTL- Enter truePRO number \nin PRO field. For Reverse Logistics call 877-402-2435.\nCUSTOMER ORDER NUMBER\n# PKGS\nCUSTOMER ORDER INFORMATION\nPallet/Slip\nWEIGHT (L

In [72]:
df

,0,1
0,200,53709863\n(402) 32644363889\nFOB:\nDispatch: 1...
1,200,"OT4946069264|PO Number; 494 Mesquite, TX Plant..."


In [76]:
def get_zip_all(string):
    zip_code = ""
    pattern1 = re.compile(r' \d{5} ')
    pattern2 = re.compile(r'[A-Za-z\s]\d{5}[A-Za-z\s]')
    first_matches = pattern1.findall(string.replace('\n', ' '))
    if first_matches:
        print(first_matches)
        zip_code = first_matches
    else:
        second_matches = pattern2.findall(string.replace('\n', ' '))
        print(second_matches)
        if second_matches:
            zip_code_list = []
            for match_str in second_matches:
                zip_list = [i for i in match_str if i.isdigit()]
                zip = "".join(zip_list)
                zip_code_list.append(zip)
            zip_code = zip_code_list
        else:
            zip_code = ""
    return zip_code

In [78]:
b = get_zip_all(data['raw_text'].decode())
print(b)

[' 70065 ', ' 88973 ', ' 19716 ', ' 95207 ', ' 95206 ', ' 95903 ', ' 82697 ', ' 57933 ']
[' 70065 ', ' 88973 ', ' 19716 ', ' 95207 ', ' 95206 ', ' 95903 ', ' 82697 ', ' 57933 ']


In [ ]:
if key.startswith(PICK_UP_ADDRESS_KEY):
            if PICKUP_FLAG:
                parsed_data[PICKUP_ADDRESS] = value
                PICKUP_FLAG = False
                
        if key.startswith(DROPOFF_LOCATION_KEY):
            if DROPOFF_FLAG:
                parsed_data[DROPOFF_ADDRESS] = value
                DROPOFF_FLAG = False

In [ ]:
if PICKUP_FLAG:
        pickup_add = get_pickup_address_from_raw_text(raw_text)
        parsed_data[PICKUP_ADDRESS] = pickup_add
        
    if DROPOFF_FLAG:
        dropadd = get_dropoff_address_from_raw_text(raw_text)
        parsed_data[DROPOFF_ADDRESS] = dropadd

In [ ]:
def get_pickup_address_from_raw_text(text):
        search_key = "From"
        start_idx = text.find(search_key)
        end_idx = text.find('Ship ')
        pickup_str = text[start_idx + len(search_key):end_idx].strip()
        try:
            pickup = pickup_str.strip()
            #pickup = pickup.replace('\n', ' ')
            pickup = pickup[:-5]
            print(len(pickup))
        except:
            pickup = ''
            
        if len(pickup) < 85:
            return pickup
        else:
            return pickup[0:18]
        
    def get_dropoff_address_from_raw_text(text):
        search_key = "To:"
        start_idx = text.find(search_key)
        end_idx = text.find('Arrival')
        dropoff_str = text[start_idx + len(search_key):end_idx].strip()
        try:
            dropoff = dropoff_str.strip()
            dropoff = dropoff.replace('\n', ' ')
            #dropoff = dropoff[:-5]
            print(len(dropoff))
        except:
            dropoff = ''
            
        return dropoff 